# Exercise Sheet 1b



### Understand Computation Graphs and Backpropagation

- Compute the gradients along the given network manually.
- Compare your results to the gradients computed by pytorch's autograd

In [1]:
import numpy as np
import torch

In [2]:
simple_net = torch.nn.Sequential(
    torch.nn.Linear(4, 2, bias=False),
    torch.nn.ReLU(),
    torch.nn.Linear(2, 1, bias=False),
    torch.nn.Sigmoid()
)

simple_net.load_state_dict(torch.load('initial_model.pth'))

criterion = torch.nn.BCELoss()

In [3]:
in_features = torch.Tensor([[-2, -0.3, 1.7, 0.2]])
target = torch.Tensor([[1.]])

prediction = simple_net(in_features)

loss = criterion(prediction, target)
loss.backward()

In [4]:
simple_net[0].weight.grad

tensor([[-0.5130, -0.0770,  0.4361,  0.0513],
        [ 0.4947,  0.0742, -0.4205, -0.0495]])

In [5]:
simple_net[2].weight.grad

tensor([[-0.0709, -0.2826]])